In [1]:
import sys
import os
import pandas as pd

from bokeh.palettes import Category10


import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")


#Adjust the working folder
file_folder = globals()['_dh'][0]
wk_dir = os.path.dirname(file_folder)
os.chdir(wk_dir)
from core.feature import *
from core.estimate import *
from core.db import *
# from core.predict import *
# from core.merge import *
# from core.merge import *

from file_cache.utils.util_pandas import *
from file_cache.cache import file_cache
from file_cache.utils.util_log import *

logging.getLogger().setLevel(logging.INFO)

import argparse
sys.argv = ['program',  '-W' , '--gp_name', 'lr_bin_9', '--shift', 0]



logging.getLogger().setLevel(logging.INFO)
!hostname

['week_day', 'stationID', 'time_ex', 'bin_id', 'p', 'd', 'q', 'out_1', 'in_1', 'out_1_p', 'in_1_p', 'out_2', 'in_2', 'out_2_p', 'in_2_p', 'out_3', 'in_3', 'out_3_p', 'in_3_p']


2019-03-23 19:32:18,332 140737245463488 util_log.py[93] INFO Start the program at:LALI2-M-G0MD, 127.0.0.1, with:Load module


yes
LALI2-M-G0MD


In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score
from scipy.stats import norm, rankdata
import warnings
import gc
import os
import time
import sys
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')
from sklearn import metrics

plt.style.use('seaborn')
sns.set(font_scale=1)
pd.set_option('display.max_columns', 500)

In [3]:
path = 'input'

test_28 = path + '/Metro_testA/testA_record_2019-01-28.csv'

In [4]:
def get_dummy_feature():
    dummy = pd.read_csv(path + '/Metro_testA/testA_submit_2019-01-29.csv')
    #dummy = dummy[dummy.stationID==0]
    dummy['time_ex'] = (pd.to_datetime(dummy['startTime']) - pd.to_datetime('2019-01-29')).dt.seconds // (60 * 10)
    dummy['time'] =  pd.to_datetime(dummy['startTime'])
    
    
    #df = df_.copy()
    
    # base time
    #dummy['day']     = (pd.to_datetime(dummy['time'])-pd.to_datetime('2019-01-01')).dt.days+1
    #dummy['week']    = pd.to_datetime(dummy['time']).dt.dayofweek + 1
    #dummy['weekend'] = (pd.to_datetime(dummy.time).dt.weekday >=5).astype(int)
    dummy['hour']    = dummy['time'].dt.hour
    dummy['minute']  = 10 * (dummy['time'].dt.minute//10)
    #dummy['time'].dt.day=20
    return dummy[['stationID', 'time_ex', 'hour','minute']]

#get_dummy_feature()


# dummy = pd.read_csv(path + '/Metro_testA/testA_submit_2019-01-29.csv')
# #pd.to_datetime(dummy['startTime']).dt.date
# #pd.to_datetime(dummy['startTime'])-pd.to_datetime(dummy['startTime']).dt.date.values
# (pd.to_datetime(dummy['startTime']) - pd.to_datetime(dummy['startTime']).dt.date.values).dt.seconds // (60 * 10)

In [5]:
#pd.to_datetime('now').minute

In [6]:
@timed()
@file_cache()
def get_base_features(file_path):
    df = pd.read_csv(file_path)
    df['time'] = pd.to_datetime(df['time'])
    logger.info('Time is done')
    #df['time_ex'] = ( df['time'] - df['time'].dt.date.values).dt.seconds // (60 * 10)
    #logger.info('Time_ex is done')
    
    #df = df_.copy()
    
    # base time
    df['day']     = (df['time']-pd.to_datetime('2019-01-01')).dt.days+1
    logger.info('Day is done')
    df['week']    = df['time'].dt.dayofweek + 1
    logger.info('Week is done')
    df['weekend'] = (df['time'].dt.weekday >=5).astype(int)
    logger.info('weekend is done')
    df['hour']    = df['time'].dt.hour
    logger.info('Hour is done')
    df['minute']  = 10 * (df['time'].dt.minute//10)
    logger.info('Minute is done')
 
    # count,sum
    dummy = get_dummy_feature()
    result = df.groupby(['stationID', 'week', 'weekend', 'day', 'hour', 'minute']).status.agg(['count', 'sum']).reset_index()
    result = dummy.merge(result, on=['stationID', 'hour','minute'], how='left')
    
    result['inNums']  = result['sum'].fillna(0)
    result['outNums'] = result['count'].fillna(0) - result['sum'].fillna(0) 
    result = result.fillna(method='ffill')
    result = result.fillna(method='bfill')
  
    
    # nunique
    tmp     = df.groupby(['stationID'])['deviceID'].nunique().reset_index(name='nuni_deviceID_of_stationID')
    result  = result.merge(tmp, on=['stationID'], how='left')
    tmp     = df.groupby(['stationID','hour'])['deviceID'].nunique().reset_index(name='nuni_deviceID_of_stationID_hour')
    result  = result.merge(tmp, on=['stationID','hour'], how='left')
    tmp     = df.groupby(['stationID','hour','minute'])['deviceID'].nunique().\
                                           reset_index(name='nuni_deviceID_of_stationID_hour_minute')
    result  = result.merge(tmp, on=['stationID','hour','minute'], how='left')
    
    # in,out

    
    #
    result['day_since_first'] = result['day'] - 1 
    result.fillna(0, inplace=True)
    del result['sum'],result['count']
    
    return result



In [7]:
#tmp = get_base_features('./input/Metro_train/record_2019-01-02.csv')
#tmp.head()

In [8]:
# data_list = os.listdir(path+'/Metro_train/')
# for i in range(0, len(data_list)):
#     if data_list[i].split('.')[-1] == 'csv':
#         print(data_list[i], i)       
#         df = get_base_features(path+'/Metro_train/' + data_list[i])
#         data = pd.concat([data, df], axis=0, ignore_index=True)
#     else:
#         continue
        
@file_cache()
def get_data():
    data = get_base_features(test_28)
    data_list = os.listdir(path+'/Metro_train/')
    for i in range(0, len(data_list)):
        if data_list[i].split('.')[-1] == 'csv':
            print(data_list[i], i)
            #df = pd.read_csv()
            df = get_base_features(path+'/Metro_train/' + data_list[i])
            data = pd.concat([data, df], axis=0, ignore_index=True)
        else:
            continue
    return data

data = get_data()
all_days = sorted(data.day.drop_duplicates().values)
print(all_days)

2019-03-23 19:32:19,203 140737245463488 util_log.py[48] INFO get_data begin with(0 paras) :[], []
2019-03-23 19:32:19,485 140737245463488 util_log.py[63] INFO cost 00.3 sec:get_data([], []), return:DF:(303264, 13), end 


[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 28.0]


In [9]:
# 剔除周末,并修改为连续时间
data = data[(data.day!=5)&(data.day!=6)]
data = data[(data.day!=12)&(data.day!=13)]
data = data[(data.day!=19)&(data.day!=20)]
data = data[(data.day!=26)&(data.day!=27)]

# def fix_day(d):
#     if d in [1,2,3,4]:
#         return d
#     elif d in [7,8,9,10,11]:
#         return d - 2
#     elif d in [14,15,16,17,18]:
#         return d - 4
#     elif d in [21,22,23,24,25]:
#         return d - 6
#     elif d in [28]:
#         return d - 8
# data['day'] = data['day'].apply(fix_day)

In [10]:
tmp = data.copy()
tmp[tmp.day==1].head()

,stationID,time_ex,hour,minute,week,weekend,day,inNums,outNums,nuni_deviceID_of_stationID,nuni_deviceID_of_stationID_hour,nuni_deviceID_of_stationID_hour_minute,day_since_first
11664,0,0,0,0,2.0,0.0,1.0,0.0,0.0,18.0,0.0,0.0,0.0
11665,0,1,0,10,2.0,0.0,1.0,0.0,0.0,18.0,0.0,0.0,0.0
11666,0,2,0,20,2.0,0.0,1.0,0.0,0.0,18.0,0.0,0.0,0.0
11667,0,3,0,30,2.0,0.0,1.0,0.0,0.0,18.0,0.0,0.0,0.0
11668,0,4,0,40,2.0,0.0,1.0,0.0,0.0,18.0,0.0,0.0,0.0


In [11]:
# test = pd.read_csv(path + '/Metro_testA/testA_submit_2019-01-29.csv')
# test['week']    = pd.to_datetime(test['startTime']).dt.dayofweek + 1
# test['weekend'] = (pd.to_datetime(test.startTime).dt.weekday >=5).astype(int)
# test['day']     = test['startTime'].apply(lambda x: int(x[8:10]))
# test['hour']    = test['startTime'].apply(lambda x: int(x[11:13]))
# test['minute']  = test['startTime'].apply(lambda x: int(x[14:15]+'0'))
# test['day_since_first'] = test['day'] - 1
# test = test.drop(['startTime','endTime'], axis=1)
# data = pd.concat([data,test], axis=0, ignore_index=True)

stat_columns = ['inNums','outNums']

In [12]:
def get_refer_day(d):
    sn = [item for item in range(40) if item%7 not in [5,6] ]
    sn_map = dict(zip(sn[1:], sn[:-1]))
    return sn_map[d]
print(get_refer_day(2))

# Bing feature#25 to label#28

1


In [13]:

tmp = data.copy()
# tmp_df = tmp[tmp.day==1]
# tmp_df['day'] = tmp_df['day'] - 1
# print(tmp_df.shape)
# tmp = pd.concat([tmp, tmp_df], axis=0, ignore_index=True)
# print(tmp.shape)
tmp['day'] = tmp['day'].apply(get_refer_day)
#tmp.day.value_counts().sort_index(), data.day.value_counts().sort_index()



In [14]:
 


for f in stat_columns:
    tmp.rename(columns={f: f+'_next'}, inplace=True) 
    
tmp = tmp[['stationID','day','hour','minute','inNums_next','outNums_next']]

print(data.shape, tmp.shape)
data = data.merge(tmp, on=['stationID','day','hour','minute'], how='left')
data.fillna(0, inplace=True)
data[data.day==25]

(233280, 13) (233280, 6)


,stationID,time_ex,hour,minute,week,weekend,day,inNums,outNums,nuni_deviceID_of_stationID,nuni_deviceID_of_stationID_hour,nuni_deviceID_of_stationID_hour_minute,day_since_first,inNums_next,outNums_next
221616,0,0,0,0,5.0,0.0,25.0,0.0,2.0,18.0,4.0,2.0,24.0,0.0,1.0
221617,0,1,0,10,5.0,0.0,25.0,0.0,3.0,18.0,4.0,2.0,24.0,0.0,2.0
221618,0,2,0,20,5.0,0.0,25.0,0.0,0.0,18.0,4.0,0.0,24.0,0.0,0.0
221619,0,3,0,30,5.0,0.0,25.0,0.0,0.0,18.0,4.0,0.0,24.0,0.0,0.0
221620,0,4,0,40,5.0,0.0,25.0,0.0,0.0,18.0,4.0,0.0,24.0,0.0,0.0
221621,0,5,0,50,5.0,0.0,25.0,0.0,0.0,18.0,4.0,0.0,24.0,0.0,0.0
221622,0,6,1,0,5.0,0.0,25.0,0.0,0.0,18.0,0.0,0.0,24.0,0.0,0.0
221623,0,7,1,10,5.0,0.0,25.0,0.0,0.0,18.0,0.0,0.0,24.0,0.0,0.0
221624,0,8,1,20,5.0,0.0,25.0,0.0,0.0,18.0,0.0,0.0,24.0,0.0,0.0
221625,0,9,1,30,5.0,0.0,25.0,0.0,0.0,18.0,0.0,0.0,24.0,0.0,0.0


In [15]:
tmp = data.groupby(['stationID','week','hour','minute'], as_index=False)['inNums'].agg({
                                                                        'inNums_whm_max'    : 'max',
                                                                        'inNums_whm_min'    : 'min',
                                                                        'inNums_whm_mean'   : 'mean'
                                                                        })
data = data.merge(tmp, on=['stationID','week','hour','minute'], how='left')

tmp = data.groupby(['stationID','week','hour','minute'], as_index=False)['outNums'].agg({
                                                                        'outNums_whm_max'    : 'max',
                                                                        'outNums_whm_min'    : 'min',
                                                                        'outNums_whm_mean'   : 'mean'
                                                                        })
data = data.merge(tmp, on=['stationID','week','hour','minute'], how='left')

tmp = data.groupby(['stationID','week','hour'], as_index=False)['inNums'].agg({
                                                                        'inNums_wh_max'    : 'max',
                                                                        'inNums_wh_min'    : 'min',
                                                                        'inNums_wh_mean'   : 'mean'
                                                                        })
data = data.merge(tmp, on=['stationID','week','hour'], how='left')

tmp = data.groupby(['stationID','week','hour'], as_index=False)['outNums'].agg({
                                                                        #'outNums_wh_max'    : 'max',
                                                                        #'outNums_wh_min'    : 'min',
                                                                        'outNums_wh_mean'   : 'mean'
                                                                        })
data = data.merge(tmp, on=['stationID','week','hour'], how='left')

In [16]:
#data=data[data.day!=29]
all_days = sorted(data.day.drop_duplicates().values)
print(all_days)

[1.0, 2.0, 3.0, 4.0, 7.0, 8.0, 9.0, 10.0, 11.0, 14.0, 15.0, 16.0, 17.0, 18.0, 21.0, 22.0, 23.0, 24.0, 25.0, 28.0]


In [17]:
data[data.day==29]

,stationID,time_ex,hour,minute,week,weekend,day,inNums,outNums,nuni_deviceID_of_stationID,nuni_deviceID_of_stationID_hour,nuni_deviceID_of_stationID_hour_minute,day_since_first,inNums_next,outNums_next,inNums_whm_max,inNums_whm_min,inNums_whm_mean,outNums_whm_max,outNums_whm_min,outNums_whm_mean,inNums_wh_max,inNums_wh_min,inNums_wh_mean,outNums_wh_mean


# Validate from here

In [18]:
# def recover_day(d):
#     if d in [1,2,3,4]:
#         return d
#     elif d in [5,6,7,8,9]:
#         return d + 2
#     elif d in [10,11,12,13,14]:
#         return d + 4
#     elif d in [15,16,17,18,19]:
#         return d + 6
#     elif d == 20:
#         return d + 8
#     else:
#         return d

all_days = sorted(data.day.drop_duplicates().values.astype(int))
print(len(all_days), all_days)

last_day = all_days[-1]
vali_day = 21 #25 #21
test_day = 28

#predict_day = last_day + 1

all_columns = [f for f in data.columns if f not in ['weekend','inNums_next','outNums_next']]
### all data
all_data = data[data.day!=test_day]
#all_data['day'] = all_data['day'].apply(recover_day)
X_data = all_data[all_columns].values

train = data[~data.day.isin([vali_day, test_day])]
#train['day'] = train['day'].apply(recover_day)
X_train = train[all_columns].values

valid = data[data.day==vali_day]
#valid['day'] = valid['day'].apply(recover_day)
X_valid = valid[all_columns].values

test  = data[data.day==test_day]
X_test = test[all_columns].values

20 [1, 2, 3, 4, 7, 8, 9, 10, 11, 14, 15, 16, 17, 18, 21, 22, 23, 24, 25, 28]


In [19]:
data[data.day==28].sort_values(['outNums','outNums_next']).tail(10)

,stationID,time_ex,hour,minute,week,weekend,day,inNums,outNums,nuni_deviceID_of_stationID,nuni_deviceID_of_stationID_hour,nuni_deviceID_of_stationID_hour_minute,day_since_first,inNums_next,outNums_next,inNums_whm_max,inNums_whm_min,inNums_whm_mean,outNums_whm_max,outNums_whm_min,outNums_whm_mean,inNums_wh_max,inNums_wh_min,inNums_wh_mean,outNums_wh_mean
2210,15,50,8,20,1.0,0.0,28.0,719.0,1493.0,67.0,65.0,56.0,27.0,0.0,0.0,1036.0,719.0,899.00,1852.0,1493.0,1699.25,1664.0,719.0,1178.833333,1593.416667
2235,15,75,12,30,1.0,0.0,28.0,1026.0,1511.0,67.0,66.0,59.0,27.0,0.0,0.0,1160.0,888.0,1011.50,1511.0,921.0,1239.75,1180.0,734.0,974.083333,1146.541667
625,4,49,8,10,1.0,0.0,28.0,493.0,1547.0,27.0,27.0,27.0,27.0,0.0,0.0,547.0,493.0,524.00,1965.0,1547.0,1773.25,720.0,319.0,496.041667,1981.625000
2212,15,52,8,40,1.0,0.0,28.0,1206.0,1591.0,67.0,65.0,61.0,27.0,0.0,0.0,1336.0,1206.0,1274.50,1869.0,1591.0,1727.50,1664.0,719.0,1178.833333,1593.416667
2209,15,49,8,10,1.0,0.0,28.0,785.0,1622.0,67.0,65.0,57.0,27.0,0.0,0.0,1139.0,785.0,968.25,1681.0,1470.0,1585.50,1664.0,719.0,1178.833333,1593.416667
629,4,53,8,50,1.0,0.0,28.0,319.0,1665.0,27.0,27.0,27.0,27.0,0.0,0.0,530.0,319.0,377.50,1831.0,1665.0,1785.50,720.0,319.0,496.041667,1981.625000
2211,15,51,8,30,1.0,0.0,28.0,782.0,1688.0,67.0,65.0,55.0,27.0,0.0,0.0,1198.0,782.0,1048.00,1688.0,1582.0,1649.50,1664.0,719.0,1178.833333,1593.416667
627,4,51,8,30,1.0,0.0,28.0,444.0,2074.0,27.0,27.0,27.0,27.0,0.0,0.0,542.0,444.0,489.00,2384.0,2074.0,2258.75,720.0,319.0,496.041667,1981.625000
628,4,52,8,40,1.0,0.0,28.0,427.0,2324.0,27.0,27.0,27.0,27.0,0.0,0.0,444.0,367.0,410.00,2803.0,2324.0,2629.25,720.0,319.0,496.041667,1981.625000
626,4,50,8,20,1.0,0.0,28.0,436.0,2452.0,27.0,27.0,27.0,27.0,0.0,0.0,529.0,436.0,491.75,2465.0,2136.0,2309.00,720.0,319.0,496.041667,1981.625000


In [20]:
print(all_data.shape, train.shape, valid.shape)
#all_data.head()
#all_data.to_csv('./cache/output.csv',index=None)

(221616, 25) (209952, 25) (11664, 25)


In [21]:
sub_model = False

params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mae',
    'num_leaves': 63,
    'learning_rate': 0.01,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.9,
    'bagging_seed':0,
    'bagging_freq': 1,
    'verbose': 1,
    'reg_alpha':1,
    'reg_lambda':2
}

######################################################inNums
y_train = train['inNums_next']
y_valid = valid['inNums_next']
y_data  = all_data['inNums_next']

print(X_train.shape, y_train.shape, X_valid.shape, y_valid.shape)

lgb_train = lgb.Dataset(X_train, y_train)
lgb_evals = lgb.Dataset(X_valid, y_valid , reference=lgb_train)
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=90000,
                valid_sets=[lgb_train,lgb_evals],
                valid_names=['train','valid'],
                early_stopping_rounds=200,
                verbose_eval=1000,
                )
in_score = gbm.best_score['valid']['l1']
in_iter = gbm.best_iteration

if sub_model:
    ### all_data
    lgb_train = lgb.Dataset(X_data, y_data)
    gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=gbm.best_iteration,
                    valid_sets=[lgb_train],
                    valid_names=['train'],
                    verbose_eval=1000,
                    )
    test['inNums_next'] = gbm.predict(X_test)

######################################################outNums
y_train = train['outNums_next']
y_valid = valid['outNums_next']
y_data  = all_data['outNums_next']
lgb_train = lgb.Dataset(X_train, y_train)
lgb_evals = lgb.Dataset(X_valid, y_valid , reference=lgb_train)
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=90000,
                valid_sets=[lgb_train,lgb_evals],
                valid_names=['train','valid'],
                early_stopping_rounds=200,
                verbose_eval=1000,
                )
out_score = gbm.best_score['valid']['l1']
out_iter = gbm.best_iteration

if sub_model:

    ### all_data
    lgb_train = lgb.Dataset(X_data, y_data)
    gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=gbm.best_iteration,
                    valid_sets=[lgb_train],
                    valid_names=['train'],
                    verbose_eval=1000,
                    )
    test['inNums_next'] = gbm.predict(X_test)


print(f'vali_day:{vali_day},test_day:{test_day}')
print(f'in_score:{in_score:06.4f}@{in_iter}, out_score:{out_score:06.4f}@{out_iter}')

#vali_day:25,test_day:28, in_score:13.426748289406717, out_score:14.65927169267639

(209952, 22) (209952,) (11664, 22) (11664,)
Training until validation scores don't improve for 200 rounds.
[1000]	train's l1: 10.8771	valid's l1: 11.0354
[2000]	train's l1: 10.2192	valid's l1: 10.9413
[3000]	train's l1: 9.75882	valid's l1: 10.8784
[4000]	train's l1: 9.39732	valid's l1: 10.8387
[5000]	train's l1: 9.08945	valid's l1: 10.8113
[6000]	train's l1: 8.82228	valid's l1: 10.787
[7000]	train's l1: 8.5832	valid's l1: 10.7665
[8000]	train's l1: 8.36479	valid's l1: 10.7525
[9000]	train's l1: 8.16253	valid's l1: 10.7382
[10000]	train's l1: 7.97328	valid's l1: 10.7289
[11000]	train's l1: 7.79837	valid's l1: 10.7203
Early stopping, best iteration is:
[10958]	train's l1: 7.80494	valid's l1: 10.7198
Training until validation scores don't improve for 200 rounds.
[1000]	train's l1: 11.8758	valid's l1: 12.2225
[2000]	train's l1: 11.2435	valid's l1: 12.1285
[3000]	train's l1: 10.7935	valid's l1: 12.1062
[4000]	train's l1: 10.4266	valid's l1: 12.088
[5000]	train's l1: 10.1061	valid's l1: 12.0

In [22]:
if sub_model:
    sub = pd.read_csv(path + '/Metro_testA/testA_submit_2019-01-29.csv')
    sub['inNums']   = test['inNums_next'].values
    sub['outNums']  = test['inNums_next'].values
    # 结果修正
    sub.loc[sub.inNums<0 , 'inNums']  = 0
    sub.loc[sub.outNums<0, 'outNums'] = 0

    avg = (in_score+out_score)/2
    file_name = f'output/sub_{avg:06.4f}_{in_score:06.4f}_{out_score:06.4f}_{int(time.time() % 10000000)}.csv'
    logger.info(file_name)

    #13.2464/15.0891
    #13.1399/15.054/14.0970
    #14.0970_13.1399_15.0540
    sub[['stationID', 'startTime', 'endTime', 'inNums', 'outNums']].to_csv(file_name, index=False)